In [1]:
#Principales librerias
import pandas as pd
import numpy as np
import random
import time

cuits=pd.read_csv('df_cuits.csv') #Si tienen el archivo guardado en local drive mandan el path ahí en vez del nombre del file. 
cuits=cuits.cuit_empleador.unique()

test_cuits = cuits

#Librerias web requests y threading
import requests
from bs4 import BeautifulSoup
#from time import sleep
import threading
from tqdm import tqdm
import re

#Fijamos la cantidad de hilos (threadings)
n_threads = 5 #fijo cinco pero se puede poner cualquier valor entre 1 y len(cuits)


#Vamos a determinar la cantidad de elementos que va a tener que analizar cada thread. 
p=len(test_cuits)//n_threads # Acá hacemos división entera para que me divida la cantidad de elementos en n partes iguales.
inicios = []
fines = []
inicio=0
fin=p


#Acá vamos a crear los inicios y los fines. 
for i in range(n_threads):
  inicios.append(inicio)
  fines.append(fin)
  inicio= inicio + p 
  fin= fin + p

cuits_and_codes = []

def get_codes(inicio, fin): 
 
  #Fijamos headers
  headers = {'User-Agent': 
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
       
  #h=0 #Indexador para armar lista de tuplas

  #Loop a través de la lista de links
  for i in tqdm(range(inicio, fin)):
    
    cuit = test_cuits[i]
    #Obtención de datos del links[i]. 
    page='https://www.cuitonline.com/detalle/'+str(cuit)+'/'
    
    tree = requests.get(page, headers = headers, timeout=10)
    
    soup = BeautifulSoup(tree.content, 'html.parser')
    
    txt = list(soup.find_all('ul'))[8].text
    
    acts_tup = re.findall('Actividades: (.*?) [a-z]|Actividades: (.*?)$|Actividad: (.*?)$', txt)[0]

    acts = ",".join(acts_tup)
    
    codes = [x.replace("]","").replace(" ","") for x in re.findall('] [0-9]+ -',acts)]
    
    if len(codes)==0: 
        cuits_and_codes.append((cuit, np.nan))
    else: 
        for c in codes: 
            cuits_and_codes.append((cuit, str(c).zfill(6)))
    
    time.sleep(random.uniform(0.2,0.7))


  return cuits_and_codes
  
#Acá le pido al módulo threading que me ejecute la función en threadings

threads=[]
for i in range(len(inicios)):
  t=threading.Thread(target=get_codes, args=(inicios[i], fines[i],))
  threads.append(t)
  t.start()

for t in threads:
  t.join()


#Armo el DataFrame
df=pd.DataFrame(cuits_and_codes, columns=['cuit','codes'])#.explode('codes').reset_index(drop=True)

#Exporto a CSV
df.to_csv('cuits_y_codigos.csv', sep=',', index=False)       
                                                             


  0%|                                                                                         | 0/2375 [00:00<?, ?it/s]


  0%|                                                                                         | 0/2375 [00:00<?, ?it/s]

  0%|                                                                                         | 0/2375 [00:00<?, ?it/s]



  0%|                                                                                         | 0/2375 [00:00<?, ?it/s]


  0%|                                                                                 | 1/2375 [00:01<52:39,  1.33s/it]

  0%|                                                                                 | 1/2375 [00:01<54:56,  1.39s/it]



  0%|                                                                               | 1/2375 [00:01<1:04:24,  1.63s/it]



  0%|                                                                                 | 2/2375 [00:02<54:52,  1.39s/it]


  0%|                 

In [7]:
df[df.codes.isna()==False].to_csv('cuits_y_codigos.csv', index=False)